In [1]:
import cv2 as cv
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calibrate_camera(images_folder):
    images_names = glob.glob(images_folder)
    print(images_names)
    images = []
    for imname in images_names:
        im = cv.imread(imname, 1)
        images.append(im)

    #criteria used by checkerboard pattern detector.
    #Change this if the code can't find the checkerboard
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
    rows = 6 #number of checkerboard rows.
    columns = 9 #number of checkerboard columns.
    world_scaling = 22.8 #mm #change this to the real world square size. Or not.
 
    #coordinates of squares in the checkerboard world space
    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp
 
    #frame dimensions. Frames should be the same size.
    width = images[0].shape[1]
    height = images[0].shape[0]
 
    #Pixel coordinates of checkerboards
    imgpoints = [] # 2d points in image plane.
 
    #coordinates of the checkerboard in checkerboard world space.
    objpoints = [] # 3d point in real world space
 
 
    for frame in images:
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
 
        #find the checkerboard
        ret, corners = cv.findChessboardCorners(gray, (rows, columns), None)
 
        if ret == True:
 
            #Convolution size used to improve corner detection. Don't make this too large.
            conv_size = (11, 11)
 
            #opencv can attempt to improve the checkerboard coordinates
            corners = cv.cornerSubPix(gray, corners, conv_size, (-1, -1), criteria)
            cv.drawChessboardCorners(frame, (rows,columns), corners, ret)
            # plt.imshow(frame, cmap='gray')
            # plt.show()
 
            objpoints.append(objp)
            imgpoints.append(corners)
 
 
 
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, (width, height), None, None)
    print('rmse:', ret)
    print('camera matrix:\n', mtx)
    print('distortion coeffs:', dist)
    print('Rs:\n', rvecs)
    print('Ts:\n', tvecs)
 
    return mtx, dist

In [3]:
def stereo_calibrate(mtx1, dist1, mtx2, dist2, frames_folder1, frames_folder2):
    #read the synched frames
    images_names1 = glob.glob(frames_folder1)
    images_names2 = glob.glob(frames_folder2)
    c1_images_names = sorted(images_names1)
    c2_images_names = sorted(images_names1)

    print(c1_images_names)
    print(c2_images_names)
    c1_images = []
    c2_images = []
    for im1, im2 in zip(c1_images_names, c2_images_names):
        _im = cv.imread(im1, 1)
        c1_images.append(_im)
 
        _im = cv.imread(im2, 1)
        c2_images.append(_im)

 
    #change this if stereo calibration not good.
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
 
    rows = 6 #number of checkerboard rows.
    columns = 9 #number of checkerboard columns.
    world_scaling = 22.8 #change this to the real world square size. Or not.
 
    #coordinates of squares in the checkerboard world space
    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp
 
    #frame dimensions. Frames should be the same size.
    width = c1_images[0].shape[1]
    height = c1_images[0].shape[0]
 
    #Pixel coordinates of checkerboards
    imgpoints_left = [] # 2d points in image plane.
    imgpoints_right = []
 
    #coordinates of the checkerboard in checkerboard world space.
    objpoints = [] # 3d point in real world space
 
    for frame1, frame2 in zip(c1_images, c2_images):

        gray1 = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
        gray2 = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
        c_ret1, corners1 = cv.findChessboardCorners(gray1, (6, 9), None)
        print(c_ret1)
        c_ret2, corners2 = cv.findChessboardCorners(gray2, (6, 9), None)
        print(c_ret2)
        
        if c_ret1 == True and c_ret2 == True:
            corners1 = cv.cornerSubPix(gray1, corners1, (11, 11), (-1, -1), criteria)
            corners2 = cv.cornerSubPix(gray2, corners2, (11, 11), (-1, -1), criteria)
 
            cv.drawChessboardCorners(frame1, (6,9), corners1, c_ret1)
            # plt.imshow(frame1, cmap='gray')
            # plt.show()
 
            cv.drawChessboardCorners(frame2, (6,9), corners2, c_ret2)
            
 
            objpoints.append(objp)
            imgpoints_left.append(corners1)
            imgpoints_right.append(corners2)
 
    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC
    ret, CM1, dist1, CM2, dist2, R, T, E, F = cv.stereoCalibrate(objpoints, imgpoints_left, imgpoints_right, mtx1, dist1,
                                                                 mtx2, dist2, (width, height), criteria = criteria, flags = stereocalibration_flags)
 
    print(ret)
    return R, T

In [4]:
mtx1, dist1 = calibrate_camera(images_folder = 'cam1/*')
mtx2, dist2 = calibrate_camera(images_folder = 'cam2/*')
 
R, T = stereo_calibrate(mtx1, dist1, mtx2, dist2, 'cam1/*', 'cam2/*')

['cam1\\1.jpg', 'cam1\\2.jpg', 'cam1\\3.jpg', 'cam1\\4.jpg', 'cam1\\5.jpg', 'cam1\\6.jpg', 'cam1\\7.jpg', 'cam1\\8.jpg', 'cam1\\9.jpg']
rmse: 0.2830134567689578
camera matrix:
 [[864.69624261   0.         373.75411988]
 [  0.         863.87899616 284.20528188]
 [  0.           0.           1.        ]]
distortion coeffs: [[-4.12694537e-01  2.05199842e-02  5.02708563e-04  5.79529209e-05
   5.68885605e-01]]
Rs:
 (array([[ 0.03450909],
       [-0.04877535],
       [-1.6224199 ]]), array([[ 0.20898974],
       [ 0.21212664],
       [-1.63571167]]), array([[ 0.15115678],
       [ 0.09243283],
       [-1.27764291]]), array([[ 0.16136022],
       [ 0.16782974],
       [-1.55948546]]), array([[ 0.26973711],
       [-0.21644071],
       [-1.65316626]]), array([[ 0.17715226],
       [-0.05693562],
       [-1.56271648]]), array([[ 0.26213559],
       [ 0.06626607],
       [-1.5607413 ]]), array([[ 0.36359623],
       [ 0.00548695],
       [-1.64185202]]), array([[ 0.31911452],
       [ 0.283424  